In [1]:
from sklearn.metrics import classification_report, confusion_matrix
from keras import layers
from keras.models import Sequential
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import io
from math import sqrt
import numpy as np
import pandas as pd
import time
import os, sys
import pyedflib
import urllib.request

Using TensorFlow backend.


In [2]:
def takehalfnums(matrix):
    halfnums=[]
    length = len(matrix)
    for i in range(length):
        for j in range(i):
            halfnums.append(matrix[i][j])
    return np.array(halfnums)

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    # classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    #plt.rcParams["figure.figsize"] = (14,14)
    fig, ax = plt.subplots()
    
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    #fig.tight_layout()
    fig.subplots_adjust(top=1)
    fig.show()
    return ax

In [3]:
MMI_SUBJECT=80
MMI_TRIAL = 14
MMI_CHANNEL = 64
MMI_SAMPLERATE = 160
MMI_FEATURE_NUM = 2016
MMI_SEQUENCE_LEN = 10

## TRIAL 1 (8개 단위로 평균)

In [5]:
file_path='./eegmmidb_edf/S'
save_path='D:/DEAP dataset/EEGMMIDB/eegmmidb_feature/S'

temp = np.zeros((64,8))
temp2 = np.zeros((64,64))
MMI_half_num = np.zeros((1,MMI_FEATURE_NUM))
MMI_seq_data = np.zeros((1,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM))

for i in tqdm(range(MMI_SUBJECT)):
    MMI_data_EyeClose = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_EyeOpen = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_RightHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_LeftHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_BothHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_BothFeet = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    
    for j in range(MMI_TRIAL):
        idx = 0
        
        path = file_path+str(i+1).zfill(3)+'/s'+str(i+1).zfill(3)+'R'+str(j+1).zfill(2)+'.edf'
        f = pyedflib.EdfReader(path)
        annotations = f.readAnnotations()
        buf_len = f.getNSamples()[0]
        sigbufs = np.zeros((MMI_CHANNEL, buf_len))
        for k in range(MMI_CHANNEL):
            sigbufs[k,:] = f.readSignal(k)
        f = pyedflib.close_file
        
        time_stamp = annotations[0]
        label = annotations[2]
                
        if (j == 1 or j == 2):
            for m in range((int)(buf_len/64 - 2)): #뒤쪽 부분 모두 0이라 제외
                for n in range(8):
                    temp[:,n] = np.mean(sigbufs[:,(m*64)+(n*8) : (m*64)+((n+1)*8)], axis = 1)
                temp2 = np.corrcoef(temp)
                
                MMI_half_num = takehalfnums(temp2)
                MMI_seq_data[0,idx,:] = MMI_half_num
                idx += 1
                
                if(idx == 10):
                    if(j==1):
                        MMI_data_EyeOpen = np.concatenate((MMI_data_EyeOpen, MMI_seq_data), axis = 0)
                    else:
                        MMI_data_EyeClose = np.concatenate((MMI_data_EyeClose, MMI_seq_data), axis = 0)
                    idx = 0
            
        elif (j%2 == 0 and j>2):
        
            for k in range(len(label)):
                for m in range(10):
                    for n in range(8):
                        temp[:,n] = np.mean(sigbufs[:,int(time_stamp[k])*MMI_SAMPLERATE+(m*64)+(n*8) : 
                                                       int(time_stamp[k])*MMI_SAMPLERATE+(m*64)+((n+1)*8)], axis = 1)
                    temp2 = np.corrcoef(temp)

                    MMI_half_num = takehalfnums(temp2)
                    MMI_seq_data[0,idx,:] = MMI_half_num
                    idx += 1

                    if(idx == 10):
                        if(label[k] == 'T1'):
                            if(j%4 == 0):
                                MMI_data_LeftHand = np.concatenate((MMI_data_LeftHand, MMI_seq_data), axis = 0)
                            else:
                                MMI_data_BothHand = np.concatenate((MMI_data_BothHand, MMI_seq_data), axis = 0)
                        elif(label[k] == 'T2'):
                            if(j%4 == 0):
                                MMI_data_RightHand = np.concatenate((MMI_data_RightHand, MMI_seq_data), axis = 0)
                            else:
                                MMI_data_BothFeet = np.concatenate((MMI_data_BothFeet, MMI_seq_data), axis = 0)
                        idx = 0
    path = save_path+str(i+1).zfill(3)
    print(path)
    np.save(path+'/EyeClose', MMI_data_EyeClose)
    np.save(path+'/EyeOpen', MMI_data_EyeOpen)
    np.save(path+'/BothHand', MMI_data_BothHand)
    np.save(path+'/BothFeet', MMI_data_BothFeet)
    np.save(path+'/RightHand', MMI_data_RightHand)
    np.save(path+'/LeftHand', MMI_data_LeftHand)

print(MMI_data_EyeOpen.shape)
print(MMI_data_EyeClose.shape)
print(MMI_data_BothFeet.shape)
print(MMI_data_BothHand.shape)
print(MMI_data_RightHand.shape)
print(MMI_data_LeftHand.shape)

print("Feature extract completed")

  0%|                                                                                          | 0/109 [00:00<?, ?it/s]

D:/DEAP dataset/EEGMMIDB/eegmmidb_feature/S001


  1%|▊                                                                                 | 1/109 [00:05<09:17,  5.16s/it]

D:/DEAP dataset/EEGMMIDB/eegmmidb_feature/S002


  2%|█▌                                                                                | 2/109 [00:10<09:20,  5.24s/it]

...

D:/DEAP dataset/EEGMMIDB/eegmmidb_feature/S108


 99%|███████████████████████████████████████████████████████████████████████████████▎| 108/109 [08:31<00:04,  4.08s/it]

D:/DEAP dataset/EEGMMIDB/eegmmidb_feature/S109


100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [08:35<00:00,  4.10s/it]


(15, 10, 2016)
(30, 10, 2016)
(15, 10, 2016)
(15, 10, 2016)
(21, 10, 2016)
(24, 10, 2016)
Feature extract completed


##  TRIAL 2 (8개 단위로 평균, sliding window)

In [6]:
file_path='./eegmmidb_edf/S'
save_path='D:/DEAP dataset/EEGMMIDB/eegmmidb_feature/S'

temp = np.zeros((64,8))
temp2 = np.zeros((64,64))
MMI_half_num = np.zeros((1,MMI_FEATURE_NUM))
MMI_seq_data = np.zeros((1,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM))

for i in tqdm(range(MMI_SUBJECT)):
    MMI_data_EyeClose = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_EyeOpen = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_RightHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_LeftHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_BothHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    MMI_data_BothFeet = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
    
    for j in range(MMI_TRIAL):
        idx = 0
        
        path = file_path+str(i+1).zfill(3)+'/s'+str(i+1).zfill(3)+'R'+str(j+1).zfill(2)+'.edf'
        f = pyedflib.EdfReader(path)
        annotations = f.readAnnotations()
        buf_len = f.getNSamples()[0]
        sigbufs = np.zeros((MMI_CHANNEL, buf_len))
        for k in range(MMI_CHANNEL):
            sigbufs[k,:] = f.readSignal(k)
        f = pyedflib.close_file
        
        ##prevent mean of empty slice
        train_data = np.nan_to_num(sigbufs)
        
        time_stamp = annotations[0]
        label = annotations[2]
                
        if (j == 1 or j == 2):
            for m in range((int)(buf_len/8 - 20)): #뒤쪽 부분 모두 0이라 제외
                for n in range(8):
                    temp[:,n] = np.mean(sigbufs[:,(m*8)+(n*8) : (m*8)+(n+1)*8], axis = 1)
                temp2 = np.corrcoef(temp)
                
                MMI_half_num = takehalfnums(temp2)
                MMI_seq_data[0,idx,:] = MMI_half_num
                idx += 1
                
                if(idx == 10):
                    if(j==1):
                        MMI_data_EyeOpen = np.concatenate((MMI_data_EyeOpen, MMI_seq_data), axis = 0)
                    else:
                        MMI_data_EyeClose = np.concatenate((MMI_data_EyeClose, MMI_seq_data), axis = 0)
                    idx = 0
            
        elif (j%2 == 0 and j>2):
        
            for k in range(len(label)):
                for m in range(72):
                    for n in range(8):
                        temp[:,n] = np.mean(sigbufs[:,int(time_stamp[k])*MMI_SAMPLERATE+(m*8)+(n*8) : 
                                                       int(time_stamp[k])*MMI_SAMPLERATE+(m*8)+((n+1)*8)], axis = 1)
                    temp2 = np.corrcoef(temp)

                    MMI_half_num = takehalfnums(temp2)
                    MMI_seq_data[0,idx,:] = MMI_half_num
                    idx += 1

                    if(idx == 10):
                        if(label[k] == 'T1'):
                            if(j%4 == 0):
                                MMI_data_LeftHand = np.concatenate((MMI_data_LeftHand, MMI_seq_data), axis = 0)
                            else:
                                MMI_data_BothHand = np.concatenate((MMI_data_BothHand, MMI_seq_data), axis = 0)
                        elif(label[k] == 'T2'):
                            if(j%4 == 0):
                                MMI_data_RightHand = np.concatenate((MMI_data_RightHand, MMI_seq_data), axis = 0)
                            else:
                                MMI_data_BothFeet = np.concatenate((MMI_data_BothFeet, MMI_seq_data), axis = 0)
                        idx = 0
    path = save_path+str(i+1).zfill(3)
    np.save(path+'/EyeClose_4', MMI_data_EyeClose)
    np.save(path+'/EyeOpen_4', MMI_data_EyeOpen)
    np.save(path+'/BothHand_4', MMI_data_BothHand)
    np.save(path+'/BothFeet_4', MMI_data_BothFeet)
    np.save(path+'/RightHand_4', MMI_data_RightHand)
    np.save(path+'/LeftHand_4', MMI_data_LeftHand)

print(MMI_data_EyeOpen.shape)
print(MMI_data_EyeClose.shape)
print(MMI_data_BothFeet.shape)
print(MMI_data_BothHand.shape)
print(MMI_data_RightHand.shape)
print(MMI_data_LeftHand.shape)

print("Feature extract completed")


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [32:18<00:00, 24.01s/it]

(120, 10, 2016)
(244, 10, 2016)
(107, 10, 2016)
(109, 10, 2016)
(167, 10, 2016)
(157, 10, 2016)
Feature extract completed


## LEARNING

In [4]:
##load the data##
load_path = './eegmmidb_feature/S'

MMI_data_EyeClose = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
MMI_data_RightHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
MMI_data_LeftHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
MMI_data_BothHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
MMI_data_BothFeet = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)

for i in tqdm(range(MMI_SUBJECT)):
    path = load_path+str(i+1).zfill(3)+'/'
    temp = np.load(path+'BothFeet_4.npy')
    MMI_data_BothFeet = np.concatenate((MMI_data_BothFeet, temp), axis = 0)
    temp = np.load(path+'BothHand_4.npy')
    MMI_data_BothHand = np.concatenate((MMI_data_BothHand, temp), axis = 0)
    temp = np.load(path+'EyeClose_4.npy')
    MMI_data_EyeClose = np.concatenate((MMI_data_EyeClose, temp), axis = 0)
    temp = np.load(path+'LeftHand_4.npy')
    MMI_data_LeftHand = np.concatenate((MMI_data_LeftHand, temp), axis = 0)
    temp = np.load(path+'RightHand_4.npy')
    MMI_data_RightHand = np.concatenate((MMI_data_RightHand, temp), axis = 0)
  

print(MMI_data_BothFeet.shape)
print(MMI_data_BothHand.shape)
print(MMI_data_EyeClose.shape)
print(MMI_data_RightHand.shape)
print(MMI_data_LeftHand.shape)

 92%|███████████████████████████████████████████████████████████████████████████▊      | 74/80 [02:29<00:20,  3.40s/it]

MemoryError: 

In [20]:
train_target = []

for i in range(MMI_data_EyeClose.shape[0]):
    train_target.append([1,0,0,0,0])
for i in range(MMI_data_BothFeet.shape[0]):
    train_target.append([0,1,0,0,0])
for i in range(MMI_data_BothHand.shape[0]):
    train_target.append([0,0,1,0,0])
for i in range(MMI_data_LeftHand.shape[0]):
    train_target.append([0,0,0,1,0])
for i in range(MMI_data_RightHand.shape[0]):
    train_target.append([0,0,0,0,1])
train_target = np.array(train_target)
    

train_data = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)

train_data = np.concatenate((train_data, MMI_data_EyeClose), axis = 0)
print(train_data.shape, MMI_data_EyeClose.shape)
MMI_data_EyeClose = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)

train_data = np.concatenate((train_data, MMI_data_BothFeet), axis = 0)
print(train_data.shape, MMI_data_BothFeet.shape)
MMI_data_BothFeet = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)

train_data = np.concatenate((train_data, MMI_data_BothHand), axis = 0)
print(train_data.shape, MMI_data_BothHand.shape)
MMI_data_BothHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)

train_data = np.concatenate((train_data, MMI_data_LeftHand), axis = 0)
print(train_data.shape, MMI_data_LeftHand.shape)
MMI_data_LeftHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)

train_data = np.concatenate((train_data, MMI_data_RightHand), axis = 0)
print(train_data.shape, MMI_data_RightHand.shape)
MMI_data_RightHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)

idx = [i for i in range(train_data.shape[0])]
np.random.shuffle(idx)

train_data = train_data[idx]
train_target = train_target[idx]


print(train_data.shape)
print(train_target.shape)

ValueError: all the input arrays must have same number of dimensions

In [ ]:
## save memory
MMI_data_EyeClose = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
MMI_data_RightHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
MMI_data_LeftHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
MMI_data_BothHand = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)
MMI_data_BothFeet = np.array([]).reshape(0,MMI_SEQUENCE_LEN,MMI_FEATURE_NUM)

In [21]:
model = Sequential()
model.add(layers.LSTM(20, return_sequences=True))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(20))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_target, epochs=150, batch_size=128, 
                    validation_split=0.2, verbose = 2) #(test_data, test_new), verbose=0)
val_acc = history.history['val_acc']
avg.append(val_acc)
print(s,max(val_acc))

W0918 21:07:19.733253  3532 deprecation_wrapper.py:119] From C:\Users\jae\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0918 21:07:19.847187  3532 deprecation_wrapper.py:119] From C:\Users\jae\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0918 21:07:19.862165  3532 deprecation_wrapper.py:119] From C:\Users\jae\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0918 21:07:19.864166  3532 deprecation_wrapper.py:119] From C:\Users\jae\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0918 21:07:20.090048  3532 deprecation_wrapper.py:119] From C:\Users\jae\Anaconda3\lib\

ValueError: Error when checking target: expected dense_1 to have shape (5,) but got array with shape (1,)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.clf()

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
print(max(val_acc))

In [ ]:
target_names = ['EyeClose','BothHand','BothFeet','Right Hand','Left Hand']
y_pred = model.predict(train_data[:100])
y_pred = np.argmax(y_pred, axis=1)
test_labels = np.argmax(train_target[:100], axis=1)
plot_confusion_matrix(test_labels, y_pred, traget_names, normalize = True)

In [13]:
## test block
for i in range(5, 10):
    print(i)

5
6
7
8
9
